# Case Study - The Current - Part 2

* The Current is an alternative radio station
* We will pull information about the play list.

# Step 0 - Insert current progress

Copy over all the relevant code from part 1 of the lab.

http://www.thecurrent.org/playlist/2014-01-01/01

In [2]:
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, strip
from composable import from_toolz as tlz
from toolz import get
from composable import pipeable
import re

In [3]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.thecurrent.org/playlist/2014-01-01/01')
current = BeautifulSoup(r.content, 'html.parser')

# Pull off the song start time

1. Inspect the element
    1. This one is tricky
    2. Time tag does not have a tag, but
    3. The surrounding div does have a class
2. Identify the html tag and class
3. Use `find_all` to make a list of all relevant tags
4. Pull off an example case
5. Write a function that extracts the start time.
6. Write a single pipe to extract the start time.
7. Confirm you have the right number of times.
8. Package your code in a function called `get_start_time`

In [4]:
get_start_time = pipeable(lambda soup: soup
 >>find_all('div', attrs={'class':'two columns songTime'})
 >>map(get_text)
 >>map(strip)
)

In [5]:
get_start_time(current)

['1:59',
 '1:54',
 '1:51',
 '1:46',
 '1:44',
 '1:38',
 '1:34',
 '1:31',
 '1:27',
 '1:23',
 '1:19',
 '1:13',
 '1:09',
 '1:05',
 '1:03',
 '1:01']

# Pull address of the album art image address

Follow a similar process to pull off the web address of the album cover image. 


In [6]:
get_src = pipeable(lambda l: l["data-src"] if l["src"] == "" else l["src"])

get_album_art = pipeable(lambda soup: soup
 >>find_all('img', attrs={'class': re.compile('album-art')})
 >>map(get_src)
)

In [7]:
get_album_art(current)

['https://albumart.publicradio.org/mb/e2/e2749c25-c2b6-493e-a2bb-10898152bd2d_5158.jpg',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/5e/5e5c8b95-d04c-432f-8cd2-c1c8d99e6e5a_3556.jpg',
 'https://albumart.publicradio.org/mb/48/48445b64-d965-369a-af3c-8193de389fd8_3ff4.jpg',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/e9/e999c049-c65b-4c5e-ad12-5596998679c7_92f9.jpg',
 'https://albumart.publicradio.org/mb/d6/d62320e2-20c4-4589-aa76-2f8ac28447dd_e03b.jpg',
 'https://albumart.publicradio.org/mb/02/028b8602-3bde-495a-a7da-15594fc4f786_351a.jpg',
 'https://albumart.publicradio.org/mb/c9/c92f73ee-527f-42ed-a5

# Putting it all together

* Make a function for each of the previous steps
* Make an overall function
    * input is a soup
    * output is a list of lists

**Hint:** You should use `zip` to put all the information together.

In [8]:
get_artist = pipeable(lambda soup: soup
 >>find_all('h5', attrs={'class':'artist'})
 >>map(get_text)
)

get_title = pipeable(lambda soup: soup
 >>find_all('h5', attrs={'class':'title'})
 >>map(get_text)
)

def get_day(soup):
    
    remove_trail = pipeable(lambda s: s.split(",")[0])
    
    return (soup
     >>find('a', attrs={'class':'start-picker'})
     >>get_text
     >>strip
     >>remove_trail
    )

def get_djName(soup):
    
    remove_label = pipeable(lambda l: l[4:])
    
    return (soup
     >>find('h5', attrs={'class':'currentDj'})
     >>get_text
     >>strip
     >>remove_label
    )

def get_timePeriod(soup):
    
    get_timePeriodFromString = pipeable(lambda l: l[-2:])

    return (soup
     >>find('span', attrs={'class':'hour-header open'})
     >>get_text
     >>strip
     >>get_timePeriodFromString
    )

In [19]:
def get_Radio_info(soup):
    
    titles = get_title(soup)
    artists = get_artist(soup)
    covers = get_album_art(soup)
    times = get_start_time(soup)
    timePeriod = get_timePeriod(soup)
    day = get_day(soup)
    dj = get_djName(soup)
    
    radio = [list(a) for a in zip(titles,artists,covers,times)]
    radio = [[t,a,c,tm,timePeriod,day,dj] for t,a,c,tm in radio]
    
    return radio

get_Radio_info(current)

[['Holy Roller',
  'Thao and The Get Down Stay Down',
  'https://albumart.publicradio.org/mb/e2/e2749c25-c2b6-493e-a2bb-10898152bd2d_5158.jpg',
  '1:59',
  'am',
  'Wednesday',
  'Mary Lucia'],
 ['Kingdom of Rust',
  'Doves',
  '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
  '1:54',
  'am',
  'Wednesday',
  'Mary Lucia'],
 ['Black Dog',
  'Frankie Lee',
  '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
  '1:51',
  'am',
  'Wednesday',
  'Mary Lucia'],
 ['Turn It Around',
  'Lucius',
  'https://albumart.publicradio.org/mb/5e/5e5c8b95-d04c-432f-8cd2-c1c8d99e6e5a_3556.jpg',
  '1:46',
  'am',
  'Wednesday',
  'Mary Lucia'],
 ['Flavor of the Month',
  'The Posies',
  'https://albumart.publicradio.org/mb/48/48445b64-d965-369a-af3c-8193de389fd8_3ff4.jpg',
  '1:44',
  'am',
  'Wednesday',
  'Mary Lucia'],
 ['Potential Wife',
  'Strange Names',
  '/assets/album-cover-default-32217dc68a771f3